In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Import of CRUD Module
from aac_crud import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "SNHU1234"
host = "nv-desktop-services.apporto.com"
port = 30442
db_name = "AAC"
collection_name = "animals"

# Instance of the AnimalShelter class with the connection parameters
shelter = AnimalShelter(username, password, host, port, db_name, collection_name)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#Grazioso Salvare’s logo
image_filename = 'Grazioso_Salvare_Logo.png' # Image of company logo
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    # Comapny Logo added to dashboard
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'width': '200px'})),
    
    html.Center(html.B(html.H1('Grazioso Salvare Dashboard - Collin Lanier'))),
    
    html.Hr(),
    
    # Dropdown for selecting different filters
    html.Div([
        dcc.Dropdown(
            id='filter-type',
            options=[
                {'label': 'All (Reset)', 'value': 'all'}, # Shows all of the data and resets the table
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
            ],
            value='all', # Default to 'All' when first loaded
            placeholder='Select a rescue category to filter',
            style={'width' : '100%'}
        ),
    ], style={'display': 'flex', 'justify-content': 'center', 'align-items': 'center', 'margin': '10px'}),
    
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         editable=False,
                         sort_action="native",
                         sort_mode="multi",
                         row_selectable='single', #Enable single row selection
                         selected_rows=[0], #Select the first row by default
                         page_action="native",
                         page_current=0,
                         page_size=10, #Limit to 10 rows per page
                         style_table={'overflowX': 'auto'}, #Enable horizontal scroll
                         style_data_conditional=[] #Will be updated in callback

                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')] # Dropdown for rescue catergories
)
def update_dashboard(filter_type):
    query = {}  # Empty query by default

    # Handle the rescue category filter
    if filter_type and filter_type != 'all':  # Apply filters only if not 'All'
        if filter_type == 'Water Rescue':
            query = {
                'breed': {'$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']},
                'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156},  # Age filter
                'sex_upon_outcome': 'Intact Female'  # Female for Water Rescue
            }
        elif filter_type == 'Mountain or Wilderness Rescue':
            query = {
                'breed': {'$in': ['German Shepard', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']},
                'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156},  # Age filter
                'sex_upon_outcome': 'Intact Male'  # Male for Mountain/Wilderness Rescue
            }
        elif filter_type == 'Disaster or Individual Tracking':
            query = {
                'breed': {'$in': ['Doberman Pinscher', 'German Shepard', 'Golden Retriever', 'Bloodhound', 'Rottweiler']},
                'age_upon_outcome_in_weeks': {'$gte': 20, '$lte': 300},  # Age filter
                'sex_upon_outcome': 'Intact Male'  # Male for Disaster/Tracking
            }

    # Get data based on the query
    df = pd.DataFrame.from_records(shelter.read(query))

    # Drop the '_id' column if it exists
    if '_id' in df.columns:
        df.drop(columns=['_id'], inplace=True)

    return df.to_dict('records')  # Return the filtered or reset data
    
# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None or len(viewData) == 0:
        return html.Div("No data available to display") # If empty
    
    # Convert filtered data to a DataFrame
    dff = pd.DataFrame.from_dict(viewData)
    
    # Check to see if breed data exists
    if 'breed' not in dff.columns:
        return html.Div("Breed data not available.")
    
    # Pie chart based on breeds
    fig = px.pie(dff, names='breed', title='Breeds based on filtered Data')
    
    return dcc.Graph(figure=fig)
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    # Check if selected_columns is None
    if selected_columns is None:
        return[] # If no columns are selected, return an empty list
    
    # Ensure selected_columns is iterable and contains valid data
    if isinstance(selected_columns, list) and all(isinstance(col, str) for col in selected_columns):
        return [{
            'if': { 'column_id': i },
            'background_color': '#D2F3FF'
        } for i in selected_columns]
    
    return [] # Return empty list if there's and issue with data 

# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:13797/
